In [1]:
import csv
import pandas as pd
import numpy as np
import pickle

In [2]:
import nltk
import string
#Importing the stopwords
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# preprocess the data

In [3]:
Alldataset= pd.read_csv("train.csv")
Alldataset= Alldataset.iloc[:,0:9].dropna()
Alldataset

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (1,5,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,conv_id,utterance_idx,context,prompt,translated_prompt,speaker_idx,utterance,translated_utterance,selfeval
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,أتذكر ذهابي إلى الألعاب النارية مع أعز أصدقائي...,1,I remember going to see the fireworks with my ...,أتذكر أنني ذهبت لمشاهدة الألعاب النارية مع أعز...,5|5|5_2|2|5
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,أتذكر ذهابي إلى الألعاب النارية مع أعز أصدقائي...,0,Was this a friend you were in love with_comma_...,هل كان هذا الصديق الذي كنت تحبه أم مجرد أفضل ص...,5|5|5_2|2|5
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,أتذكر ذهابي إلى الألعاب النارية مع أعز أصدقائي...,1,This was a best friend. I miss her.,كان هذا أفضل صديق. اشتقت لها.,5|5|5_2|2|5
3,hit:0_conv:1,4,sentimental,I remember going to the fireworks with my best...,أتذكر ذهابي إلى الألعاب النارية مع أعز أصدقائي...,0,Where has she gone?,اين ذهبت؟,5|5|5_2|2|5
4,hit:0_conv:1,5,sentimental,I remember going to the fireworks with my best...,أتذكر ذهابي إلى الألعاب النارية مع أعز أصدقائي...,1,We no longer talk.,لم نعد نتحدث.,5|5|5_2|2|5
...,...,...,...,...,...,...,...,...,...
82709,hit:12376_conv:24752,3,embarrassed,I was out to eat with some friends. and i over...,أنا متوترة قليلاً بشأن الكائنات الفضائية في ال...,524,Laughed and got me some new pants,نعم يذكرني بالأيام الخوالي. افتقد جدتي. توفيت ...,5|5|5_5|4|4
82710,hit:12376_conv:24752,4,embarrassed,I was out to eat with some friends. and i over...,أنا متوترة قليلاً بشأن الكائنات الفضائية في ال...,298,On no. Was this at a public place?,استيقظت هذا الصباح على زوجتي تخبرني أنها حامل!,5|5|5_5|4|4
82711,hit:12376_conv:24753,1,ashamed,I failed my drivers license exam because I did...,أنا متوترة قليلاً بشأن الكائنات الفضائية في ال...,298,I failed my drivers license exam because I did...,يا هذا رائع! هذا رائع أليس كذلك؟,5|4|4_5|5|5
82712,hit:12376_conv:24753,2,ashamed,I failed my drivers license exam because I did...,أنا متوترة قليلاً بشأن الكائنات الفضائية في ال...,524,Oh no_comma_ when can you take it again?,إنه رائع جدا. لقد كنا نريد طفلًا لفترة طويلة. ...,5|4|4_5|5|5


In [4]:
Alldataset["speaker_idx"]= Alldataset["speaker_idx"].apply(lambda x: pd.to_numeric(x,errors='coerce'))
Alldataset= Alldataset.dropna()

In [5]:
Alldataset["context"].unique()

array(['sentimental', 'afraid', 'proud', 'faithful', 'terrified',
       'joyful', 'angry', 'sad', 'jealous', 'grateful', 'prepared',
       'embarrassed', 'excited', 'annoyed', 'lonely', 'ashamed', 'guilty',
       'surprised', 'nostalgic', 'confident', 'furious', 'disappointed',
       'caring', 'trusting', 'disgusted', 'anticipating', 'anxious',
       'hopeful', 'content', 'impressed', 'apprehensive', 'devastated',
       ' متخوف ', ' غاضب '], dtype=object)

In [6]:
Alldataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78792 entries, 0 to 82713
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   conv_id               78792 non-null  object 
 1   utterance_idx         78792 non-null  object 
 2   context               78792 non-null  object 
 3   prompt                78792 non-null  object 
 4   translated_prompt     78792 non-null  object 
 5   speaker_idx           78792 non-null  float64
 6   utterance             78792 non-null  object 
 7   translated_utterance  78792 non-null  object 
 8   selfeval              78792 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.0+ MB


In [7]:
count= Alldataset.groupby('context').count()
count.sort_values(by=['conv_id'])

,conv_id,utterance_idx,prompt,translated_prompt,speaker_idx,utterance,translated_utterance,selfeval
context,,,,,,,,
غاضب,1,1,1,1,1,1,1,1
متخوف,2,2,2,2,2,2,2,2
faithful,1513,1513,1513,1513,1513,1513,1513,1513
apprehensive,1862,1862,1862,1862,1862,1862,1862,1862
ashamed,2055,2055,2055,2055,2055,2055,2055,2055
trusting,2091,2091,2091,2091,2091,2091,2091,2091
sentimental,2102,2102,2102,2102,2102,2102,2102,2102
caring,2122,2122,2122,2122,2122,2122,2122,2122
devastated,2257,2257,2257,2257,2257,2257,2257,2257


In [8]:
Alldataset= Alldataset[Alldataset['context'] != ' متخوف ']
Alldataset= Alldataset[Alldataset['context'] != ' غاضب ']
# Alldataset.to_csv('Alldataset.csv')

In [9]:
count= Alldataset.groupby('context').count()
count.sort_values(by=['conv_id'])

,conv_id,utterance_idx,prompt,translated_prompt,speaker_idx,utterance,translated_utterance,selfeval
context,,,,,,,,
faithful,1513,1513,1513,1513,1513,1513,1513,1513
apprehensive,1862,1862,1862,1862,1862,1862,1862,1862
ashamed,2055,2055,2055,2055,2055,2055,2055,2055
trusting,2091,2091,2091,2091,2091,2091,2091,2091
sentimental,2102,2102,2102,2102,2102,2102,2102,2102
caring,2122,2122,2122,2122,2122,2122,2122,2122
devastated,2257,2257,2257,2257,2257,2257,2257,2257
embarrassed,2262,2262,2262,2262,2262,2262,2262,2262
content,2271,2271,2271,2271,2271,2271,2271,2271


In [10]:
#Initializing the WordNetLemmatizer
lemmer = nltk.stem.WordNetLemmatizer()
#A dictionary reference for replacing special characters
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemTokens(tokens):
       return [lemmer.lemmatize(token,'v') for token in tokens if token not in set(stopwords.words('english'))]
def LemNormalize(text):
       return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))
def LemAllTokens(tokens):
       return [lemmer.lemmatize(token,'v') for token in tokens]
def LemAllNormalize(text):
       return LemAllTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [11]:
afraid_df =  Alldataset[Alldataset['context']=="afraid"]
afraid_df = afraid_df.reset_index()
confident_df =  Alldataset[Alldataset['context']=="confident"]
confident_df = confident_df.reset_index()
surprised_df =  Alldataset[Alldataset['context']=="surprised"]
surprised_df = surprised_df.reset_index()
sad_df =  Alldataset[Alldataset['context']=="sad"]
sad_df= sad_df.reset_index()

# Creating and saving the models

In [12]:
def creatingModel(df,name):    
    tokenized_dataset=pd.concat([df]).reset_index()
    tokenized_dataset=tokenized_dataset[['prompt', 'utterance','translated_utterance']]
    tokenized_dataset.utterance=tokenized_dataset.utterance.apply(lambda row: row.replace("_comma_",""))
    tokenized_dataset.prompt=tokenized_dataset.prompt.apply(lambda row: row.replace("_comma_",""))
    utterances=tokenized_dataset['utterance'].values.tolist()
    translated_utterance=tokenized_dataset['translated_utterance'].values.tolist()
    prompts_sentences=tokenized_dataset['prompt'].values.tolist()
    dic={}
    for prompt in prompts_sentences:
        dic[prompt] = tokenized_dataset.loc[tokenized_dataset['prompt'] == prompt]['utterance'].values.tolist()
    word_vectorizer = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords.words('english'))
    allwords=(utterances+prompts_sentences)
    word_vectorizer.fit(allwords)
    prompts_vectors = word_vectorizer.transform(prompts_sentences)
    utterances_vectors = word_vectorizer.transform(utterances)
    all_word_vectors=utterances_vectors*0.6 +prompts_vectors*0.4
    pickle.dump(word_vectorizer, open(name+"_word_vectorizer.sav", "wb"))
    pickle.dump(all_word_vectors, open(name+"_all_word_vectors.pickle", "wb"))
    pickle.dump(utterances, open(name+"_utterance.pickle", "wb"))
    pickle.dump(prompts_sentences, open(name+"_prompts_sentences.pickle", "wb"))
    pickle.dump(translated_utterance, open(name+"_translated_utterance.pickle", "wb"))


In [13]:
creatingModel(afraid_df,"afraid")
creatingModel(confident_df,"confident")
creatingModel(sad_df,"sad")
creatingModel(surprised_df,"surprised")

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arent', 'couldnt', 'didnt', 'doesnt', 'dont', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shant', 'shes', 'shouldnt', 'shouldve', 'thatll', 'wasnt', 'werent', 'wont', 'wouldnt', 'youd', 'youll', 'youre', 'youve'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
